In [0]:
# 📦 Importação de bibliotecas
import pandas as pd
import requests
import plotly.express as px

# 🔽 Etapa de Extração (API)
url = "https://restcountries.com/v3.1/all?fields=name,capital,region,population,latlng"
data = requests.get(url).json()

# 🧹 Etapa de Transformação (normalização e limpeza)
df = pd.json_normalize(data)

df['country'] = df['name.common']
df['official_name'] = df['name.official']
df['capital'] = df['capital'].apply(lambda x: x[0] if isinstance(x, list) and x else 'N/A')
df['latitude'] = df['latlng'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)
df['longitude'] = df['latlng'].apply(lambda x: x[1] if isinstance(x, list) and len(x) > 1 else None)

# 🔎 Remove países sem coordenadas
df_clean = df.dropna(subset=['latitude', 'longitude'])


🌍 Mapa Interativo

In [0]:
fig = px.scatter_geo(df_clean,
                     lat='latitude',
                     lon='longitude',
                     hover_name='country',
                     hover_data={'capital': True, 'population': True},
                     size='population',
                     projection='natural earth',
                     title='🌍 Mapa Interativo de Países e População')

fig.show()


📊 Análises Exploratórias

In [0]:
# Subconjunto para análise
df_select = df[['country', 'capital', 'region', 'population', 'official_name']]




In [0]:
# 📌 População média por continente
df_select.groupby('region')['population'].mean().sort_values(ascending=False)



Out[22]: region
Asia         9.209190e+07
Africa       2.308636e+07
Americas     1.823172e+07
Europe       1.409310e+07
Oceania      1.597016e+06
Antarctic    2.860000e+02
Name: population, dtype: float64

In [0]:
# 📌 Top 10 países mais populosos
country_counts_df = df_select[['country', 'population']].sort_values(by='population', ascending=False).head(10)

country_counts_df.columns = ['Pais', 'População']

display(country_counts_df)




Pais,População
China,1402112000
India,1380004385
United States,329484123
Indonesia,273523621
Pakistan,220892331
Brazil,212559409
Nigeria,206139587
Bangladesh,164689383
Russia,144104080
Mexico,128932753


In [0]:
# 📌 Quantidade de países sem capital definida
df_select[df_select['capital'] == 'N/A'].shape[0]



Out[26]: 4

In [0]:
# 📌 Contagem de países por continente
# Agrupando e renomeando colunas
region_counts_df = df_select['region'].value_counts().reset_index()
region_counts_df.columns = ['Continente', 'Quantidade de Países']

# Ordena por quantidade
region_counts_df = region_counts_df.sort_values(by='Quantidade de Países', ascending=False).reset_index(drop=True)

region_counts_df.style.set_properties(**{'text-align': 'center'}) \
    .set_table_styles([{"selector": "th", "props": [("text-align", "center")]}]) \
    .background_gradient(subset=["Quantidade de Países"], cmap="Blues")

# Exibe como tabela em Databricks
# display(region_counts_df)


,Continente,Quantidade de Países
0,Africa,59
1,Americas,56
2,Europe,53
3,Asia,50
4,Oceania,27
5,Antarctic,5
